In [1]:
import pygame
import sys



pygame 2.3.0 (SDL 2.24.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Define your desired data structure.
from langchain_community.llms import Ollama
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI

class Craft(BaseModel):
    
    word: str = Field(description="the new word which was created")

        
class Crafter:
    def __init__(self, local=False):
        
        if local:
            llm = Ollama(model="mistral:instruct", temperature=0.6)
        else:
            llm = ChatMistralAI(mistral_api_key='', model = 'mistral-large-latest')
        parser = PydanticOutputParser(pydantic_object=Craft)

        prompt = PromptTemplate(
            template="Answer the user query.\n{format_instructions}\n{query}\n",
            input_variables=["query"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
        


        chain = prompt | llm | parser
        self.chain = chain
    
    def craft(self, word1, word2):
        while True:
            try:

               

                ans = self.chain.invoke({"query": f'''You are a AI who combines word to create meaningful words.
                For example:
                Earth + Wind = Dust
                Fire + Water = Steam
                Earth + Water = Plant
                Dust + Earth = Planet
                Plant + Steam = Tea
                Planet + Wind = Storm
                Storm + Tea = Tempest
                Plant + Tempest = Tree
                Dust + Tree = Wood
                Fire + Wood = Campfire
                Dust + Water = Mud
                Campfire + Earth = Charcoal
                Charcoal + Mud = Fossil
                Water + fire = steam
                Wind + Steam = Cloud
        
                {word1} + {word2} ='''})
                return ans.word
            except:
                continue


In [ ]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Screen settings
screen_width, screen_height = 1024, 576
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Infinite Craft')

# Colors
background_color = (255, 255, 255)
text_color = (0, 0, 0)
box_color = (200, 200, 200)
sidebar_color = (100, 100, 100)
element_color = (160, 160, 160)
element_text_color = text_color #(255, 255, 255)

# Font settings
font_size = 24
font = pygame.font.Font(None, font_size)

# Element combinations
combinations = {
    ("Fire", "Water"): "Steam",
    # Add more combinations as needed
}

# Sidebar elements
sidebar_elements = ["Fire", "Water", "Earth", "Wind", "Soil", "Seed"]
sidebar_rects = []

# Define Text Elements
elements = []

# Define TextBox class
class TextBox:
    def __init__(self, text, pos, from_sidebar=False):
        self.text = text
        self.pos = pos
        self.rect = pygame.Rect(pos[0], pos[1], 120, 40)
        self.dragging = False
        self.from_sidebar = from_sidebar

        
    def draw(self, screen):
        draw_bordered_rounded_rect(screen, self.rect, box_color, box_color, 3,5)
        #pygame.draw.rect(screen, element_color, self.rect,1,1)
        text_surface = font.render(self.text, True, element_text_color)
        screen.blit(text_surface, (self.rect.x + 10 , self.rect.y + 10))

        
    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                self.dragging = True
                self.offset_x = self.rect.x - event.pos[0]
                self.offset_y = self.rect.y - event.pos[1]

        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
            if self.from_sidebar:
                # Snap back to sidebar if it's not dragged into the main area
                if not (200 < self.rect.x < screen_width):
                    elements.remove(self)
                else:
                    self.from_sidebar = False

        elif event.type == pygame.MOUSEMOTION:
            if self.dragging:
                self.rect.x = event.pos[0] + self.offset_x
                self.rect.y = event.pos[1] + self.offset_y

# Initialize sidebar elements and their rects
def init_sidebar():
    y = 100
    sidebar_rects = []
    for element in sidebar_elements:
        rect = pygame.Rect(25, y, 150, 30)
        sidebar_rects.append(rect)
        y += 50
    return sidebar_rects

sidebar_rects = init_sidebar()

import pygame
import sys

pygame.init()

# Screen settings
screen_width, screen_height = 800, 600
screen = pygame.display.set_mode((screen_width, screen_height))

# Colors
WHITE = (255, 255, 255)
GREY = (200, 200, 200)
DARK_GREY = (100, 100, 100)

# Font settings
font = pygame.font.Font(None, 30)

# Sidebar settings
sidebar_width = 200
sidebar_content_height = 5000  # Example content height
scroll_y = 0  # Scroll position

import pygame.gfxdraw

def draw_rounded_rect(surface, rect, color, corner_radius):
    ''' Draw a rectangle with rounded corners.
    Would prefer this: 
        pygame.draw.rect(surface, color, rect, border_radius=corner_radius)
    but this option is not yet supported in my version of pygame so do it ourselves.

    We use anti-aliased circles to make the corners smoother
    '''
    if rect.width < 2 * corner_radius or rect.height < 2 * corner_radius:
        raise ValueError(f"Both height (rect.height) and width (rect.width) must be > 2 * corner radius ({corner_radius})")

    # need to use anti aliasing circle drawing routines to smooth the corners
    pygame.gfxdraw.aacircle(surface, rect.left+corner_radius, rect.top+corner_radius, corner_radius, color)
    pygame.gfxdraw.aacircle(surface, rect.right-corner_radius-1, rect.top+corner_radius, corner_radius, color)
    pygame.gfxdraw.aacircle(surface, rect.left+corner_radius, rect.bottom-corner_radius-1, corner_radius, color)
    pygame.gfxdraw.aacircle(surface, rect.right-corner_radius-1, rect.bottom-corner_radius-1, corner_radius, color)

    pygame.gfxdraw.filled_circle(surface, rect.left+corner_radius, rect.top+corner_radius, corner_radius, color)
    pygame.gfxdraw.filled_circle(surface, rect.right-corner_radius-1, rect.top+corner_radius, corner_radius, color)
    pygame.gfxdraw.filled_circle(surface, rect.left+corner_radius, rect.bottom-corner_radius-1, corner_radius, color)
    pygame.gfxdraw.filled_circle(surface, rect.right-corner_radius-1, rect.bottom-corner_radius-1, corner_radius, color)

    rect_tmp = pygame.Rect(rect)

    rect_tmp.width -= 2 * corner_radius
    rect_tmp.center = rect.center
    pygame.draw.rect(surface, color, rect_tmp)

    rect_tmp.width = rect.width
    rect_tmp.height -= 2 * corner_radius
    rect_tmp.center = rect.center
    pygame.draw.rect(surface, color, rect_tmp)
    
def draw_bordered_rounded_rect(surface, rect, color, border_color, corner_radius, border_thickness):
    if corner_radius < 0:
        raise ValueError(f"border radius ({corner_radius}) must be >= 0")

    rect_tmp = pygame.Rect(rect)
    center = rect_tmp.center

    if border_thickness:
        if corner_radius <= 0:
            pygame.draw.rect(surface, border_color, rect_tmp)
        else:
            draw_rounded_rect(surface, rect_tmp, border_color, corner_radius)

        rect_tmp.inflate_ip(-2*border_thickness, -2*border_thickness)
        inner_radius = corner_radius - border_thickness + 1
    else:
        inner_radius = corner_radius

    if inner_radius <= 0:
        pygame.draw.rect(surface, color, rect_tmp)
    else:
        draw_rounded_rect(surface, rect_tmp, color, inner_radius)

        
# Draw the sidebar
def draw_sidebar(screen, sidebar_rects, scroll_y):
    pygame.draw.rect(screen, sidebar_color, [0, 0, 200, screen_height],1,1)
    scrollable_area = pygame.Rect(0, scroll_y, sidebar_width, sidebar_content_height)

    for idx, rect in enumerate(sidebar_rects):
        rect.y  += scrollable_area.y
        #pygame.draw.rect(screen, box_color, rect,2,1)
        draw_bordered_rounded_rect(screen, rect, box_color, box_color, 3,5)
        text_surface = font.render(sidebar_elements[idx], True, text_color)
        screen.blit(text_surface, (rect.x + 5, rect.y + 5))

        
# Inside the main loop, after handling events
crafter = Crafter()
# Function to check for merging elements
def merge_elements(elements, sidebar_elements):
    for i, elem1 in enumerate(elements):
        for j, elem2 in enumerate(elements):
            if i != j:  # Ensure not checking the same element
                if elem1.rect.colliderect(elem2.rect):  # Check for collision
                    if not elem1.from_sidebar and not elem2.from_sidebar:
                        # Concatenate the text to create a new element
                        new_text = crafter.craft(elem1.text, elem2.text)
                        #new_text = elem1.text+ elem2.text
                        if new_text not in sidebar_elements:
                            sidebar_elements.append(new_text)
                        new_pos = (elem1.pos[0] , elem1.pos[1] )
                        # Create the new element and add it to the list
                        new_element = TextBox(new_text, elem1.pos)
                        new_element.rect.x = elem1.rect.x
                        new_element.rect.y = elem1.rect.y
                        elements.append(new_element)
                        # Remove the original elements
                        elements.remove(elem1)
                        elements.remove(elem2)
                        return  # Exit after merging to avoid modifying list during iteration

# Call the merge function in the main loop


# Main game loop

def main():
    offset = 0
    scroll_y= 0
    scroll_speed = 1

    clock = pygame.time.Clock()
    running = True
    running = True
    clock = pygame.time.Clock()

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            
            

            elif event.type == pygame.MOUSEBUTTONDOWN:
                
                if event.button == 1:  # Left click
                    # Check if clicked on sidebar elements
                    for idx, rect in enumerate(sidebar_rects):
                        if rect.collidepoint(event.pos):
                            elements.append(TextBox(sidebar_elements[idx], event.pos, from_sidebar=True))
                            break
            elif event.type == pygame.MOUSEWHEEL:
                # Scroll up or down
                scroll_y += event.y * 10 # Adjust scrolling speed
                # Limit scrolling
                scroll_y = min(0, max(-(sidebar_content_height - screen_height), scroll_y))


            for element in elements:
                element.handle_event(event)
          
        sidebar_rects = init_sidebar()
        
        merge_elements(elements, sidebar_elements)

        screen.fill(background_color)
        draw_sidebar(screen, sidebar_rects, scroll_y)
        

        for element in elements:
            element.draw(screen)
        

            

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()
    sys.exit()

if __name__ == '__main__':
    main()
